<a href="https://colab.research.google.com/github/huang624/Question_Answering_on_PubMed_DataSet_with_PySpark/blob/main/Question_Answering_on_PubMed_DataSet_with_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset download

In [1]:
!wget https://raw.githubusercontent.com/Franck-Dernoncourt/pubmed-rct/master/PubMed_20k_RCT/train.txt
# !wget https://www.dropbox.com/s/yh0jzpjwh5857yj/train.txt?dl=0 

--2022-04-09 18:50:41--  https://raw.githubusercontent.com/Franck-Dernoncourt/pubmed-rct/master/PubMed_20k_RCT/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29384101 (28M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]  28.02M  --.-KB/s    in 0.09s   

2022-04-09 18:50:41 (320 MB/s) - ‘train.txt’ saved [29384101/29384101]



In [2]:
ls

sample_data/  train.txt


# Data observation

In [3]:
!head -n 20 train.txt

###24293578
OBJECTIVE	To investigate the efficacy of 6 weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at 12 weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .
METHODS	A total of 125 patients with primary knee OA were randomized 1:1 ; 63 received 7.5 mg/day of prednisolone and 62 received placebo for 6 weeks .
METHODS	Outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .
METHODS	Pain was assessed using the visual analog pain scale ( 0-100 mm ) .
METHODS	Secondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and 6-min walk distance ( 6MWD ) .
METHODS	Serum levels of interleukin 1 ( IL-1 ) , IL-6 , tumor necrosis factor ( TNF ) - , and high-sensitivity C-reactive protein

# System initialization

In [4]:
! wget -O init_env.sh https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0 && \
bash init_env.sh

--2022-04-09 18:50:41--  https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh [following]
--2022-04-09 18:50:41--  https://www.dropbox.com/s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6854b27cb2977348524d8f5f34.dl.dropboxusercontent.com/cd/0/inline/BjFpTDOLXPZd1REmb_GMZagAXyaOdPAjLwvX7iPVvCnUH10V8FqzbiurqZnpwUXWmY_GA0kX-GYybgqFbQ061cSB5ISYv6uZBk_uutvro3d-V2DtEIkEHYwD65LKVly8Q_xdhULTGVPIV2KCKILJc59yJlpMosuYA_HJcjZQvigRGw/file# [following]
--2022-04-09 18:50:42--  https://uc6854b27cb2977348524d8f5f34.dl.dropboxusercontent.com/cd/0/inline/BjFpTDOLXPZd1REmb

In [5]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.9-src.zip")
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

# Read file

In [6]:
pubmed = sc.textFile("./train.txt")

In [7]:
pubmed.count()

210040

# Load model for question-answering

In [8]:
!pip install -q transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'what is the better therapy for HIV?',
    'context': 'In patients with advanced HIV disease , zidovudine appears to be more effective than didanosine as initial therapy ; however , some patients with advanced HIV disease may benefit from a change to didanosine therapy after as little as 8 to 16 weeks of therapy with zidovudine '
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

     |████████████████████████████████| 4.0 MB 33.1 MB/s 
     |████████████████████████████████| 6.5 MB 26.1 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 77 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [9]:
# context = pubmed.filter(lambda x: "cardiovascular" in x and "therapy" in x).collect()
# context = pubmed.filter(lambda x: "cardiovascular" in x and "cause" in x).collect()
# context = pubmed.filter(lambda x: "caffeine" in x and "effect" in x).collect()
# context = pubmed.filter(lambda x: "antiretroviral" in x and "risk" in x).collect()
# context = pubmed.filter(lambda x: "antiretroviral" in x).collect()

# Search keyword in context

In [10]:
context = pubmed.filter(lambda x: "HIV" in x and "therapy" in x).collect()
answers = []
for c in context:
  QA_input={'question': 'what is the therapy for the HIV?', 'context':c}
  # QA_input={'question': 'what is the risk of using antiretroviral?', 'context':c}
  answers.append(nlp(QA_input)["answer"])
  print(nlp(QA_input)["answer"])

combination antiretroviral therapy
lopinavir/ritonavir
stable antiretroviral
statin therapy
ritonavir-boosted lopinavir
lopinavir-based therapy
antiretroviral therapy
isoniazid preventive therapy
combination antiretroviral
highly active antiretroviral therapy
statin therapy
antiretroviral
antiretroviral therapy
antiretroviral
boosted protease inhibitor
antiretroviral
antiretroviral
rifabutin
antiretroviral therapy
antiretroviral therapy
molecularly targeted therapy
antiretroviral
antiretroviral therapy
antiretroviral
3-drug ( N = 14 ) or 5-drug
antiretroviral therapy
rational-emotive-behavior-based therapy
antiretroviral
antiretroviral
antiretroviral
Atorvastatin
antiretroviral therapy
combination antiretroviral therapy
antiretroviral
isoniazid preventive therapy
atazanavir/ritonavir
ATV/r monotherapy
ATV/r monotherapy
monotherapy
Tenofovir alafenamide
antiretroviral therapy
antiretroviral
combination antiretroviral therapy
pre-antiretroviral therapy
antiretroviral
antituberculosis
ant

In [11]:
result = sc.parallelize(answers).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1], ascending=False)
result.collect()

[('antiretroviral', 33),
 ('antiretroviral therapy', 22),
 ('combination antiretroviral therapy', 6),
 ('isoniazid preventive therapy', 4),
 ('Antiretroviral', 3),
 ('statin therapy', 2),
 ('rifabutin', 2),
 ('ATV/r monotherapy', 2),
 ('ribavirin', 2),
 ('lopinavir/ritonavir', 1),
 ('stable antiretroviral', 1),
 ('ritonavir-boosted lopinavir', 1),
 ('highly active antiretroviral therapy', 1),
 ('molecularly targeted therapy', 1),
 ('Atorvastatin', 1),
 ('atazanavir/ritonavir', 1),
 ('monotherapy', 1),
 ('ezetimibe + fenofibrate', 1),
 ('Dialectical therapy', 1),
 ('suppressive antiretroviral therapy', 1),
 ('second-line', 1),
 ('protease inhibitor', 1),
 ('Combination antiretroviral therapy', 1),
 ('ART', 1),
 ('PEG-IFN - / ribavirin', 1),
 ('first-line therapy', 1),
 ('tuberculosis', 1),
 ('combined antiretroviral therapy', 1),
 ('lopinavir-based therapy', 1),
 ('combination antiretroviral', 1),
 ('boosted protease inhibitor', 1),
 ('3-drug ( N = 14 ) or 5-drug', 1),
 ('rational-emoti

# Save file

In [12]:
result.coalesce(1).saveAsTextFile("result")